In [1]:
import os
import subprocess
import whisper
import openai

# 환경 변수로 설정된 OpenAI API 키
openai.api_key = os.getenv("OPENAI_API_KEY")

# 경로 설정
WAV_PATH = "./wav/learner.wav"
TRANSCRIPT_PATH = "./wav/learner.txt"
LEXICON_PATH = "./lexicon.txt"
ACOUSTIC_MODEL = "/Users/jlee/Documents/MFA/pretrained_models/acoustic/korean_mfa.zip"
ALIGNMENT_OUTPUT = "./aligned"


# 1️⃣ Whisper로 STT 수행
def transcribe_audio(wav_path):
    print("🎙️ Whisper로 STT 수행 중...")
    model = whisper.load_model("base")  # base/medium/large 선택 가능
    result = model.transcribe(wav_path, language="ko")
    print(f"📝 STT 결과: {result['text']}")

    # 텍스트 저장
    with open(TRANSCRIPT_PATH, "w", encoding="utf-8") as f:
        f.write(result["text"])
    return result["text"]


# 2️⃣ MFA 정렬 수행
def run_mfa_alignment(wav_path, transcript_path, lexicon_path, model_path, output_dir):
    print("🔧 MFA 정렬 시작...")

    # 입력 폴더 준비
    input_dir = "mfa_input"
    os.makedirs(input_dir, exist_ok=True)

    # 파일 복사
    shutil.copy(wav_path, os.path.join(input_dir, "learner.wav"))
    shutil.copy(transcript_path, os.path.join(input_dir, "learner.txt"))

    # 정렬 명령어
    command = [
        "mfa",
        "align",
        input_dir,
        lexicon_path,
        model_path,
        output_dir,
        "--clean",
        "-o",
    ]
    subprocess.run(command, check=True)
    print("✅ MFA 정렬 완료!")


# 3️⃣ GPT로 피드백 생성
def generate_feedback(whisper_text, reference_text):
    print("🤖 GPT 피드백 생성 중...")
    prompt = f"""
    다음은 학습자의 STT 전사 결과와 원어민 문장입니다.

    - 학습자(STT): "{whisper_text}"
    - 원어민 스크립트: "{reference_text}"

    학습자의 발음에서 어떤 오류가 있었는지, 어떤 단어에서 문제가 있었는지, 개선점은 무엇인지 구체적인 피드백을 작성해줘.
    문장 단위, 단어 단위로 분석해줘.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4", messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


# 4️⃣ 전체 실행
if __name__ == "__main__":
    import shutil

    # 1. Whisper STT
    whisper_text = transcribe_audio(WAV_PATH)

    # 2. MFA 정렬 (사전 준비된 lexicon.txt와 모델 필요)
    run_mfa_alignment(
        WAV_PATH, TRANSCRIPT_PATH, LEXICON_PATH, ACOUSTIC_MODEL, ALIGNMENT_OUTPUT
    )

    # 3. 원어민 문장 불러오기 (비교용)
    native_script = "영화 '부산행'을 봤어요. 이 영화는 연상호 감독의 좀비 영화예요. 이 영화는 연상호 감독이 처음으로 만든 실사 영화예요. 이 감독은 이 영화를 만들기 전에는 항상 애니메이션을 만들었어요. 그 애니메이션 영화들이 너무 좋아서, 이번에 새로운 영화에 대해서 기대를 많이 했어요. 저는 원래 좀비 영화를 좋아하지 않아요. 그런데 이 영화는 정말 재미있었어요."

    # 4. GPT 피드백 생성
    feedback = generate_feedback(whisper_text, native_script)
    print("\n📣 발음 피드백:\n")
    print(feedback)

🎙️ Whisper로 STT 수행 중...


/Users/jlee/anaconda3/envs/Koach/lib/python3.8/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=

📝 STT 결과:  4화 푸사 내놓을 봤어요. 이 4화는 435가 묻어기, 준비 4화예요. 이 4화는 435가 묻어기, 조금으로 만듭실사 4화예요. 이 가무도근 이 4화 룰 만들기, 전에는 항상 애니메이션을 만들었어요. 그 애니메이션 4화 들이 너무 좋아서 이번에 새로운 4화에 대해소 기대를 많이 했어요. 저는 올해 준비 용화 룰을 좋아하지 않아요. 그런데 이 용화는 정말 재미있었어요.
🔧 MFA 정렬 시작...


 INFO     Setting up corpus information...                                      
 INFO     Loading corpus from source files...                                   


   1% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/100  [ 0:00:01 < -:--:-- , ? it/s ]
   0% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/1  [ 0:00:00 < -:--:-- , ? it/s ]

...

 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:01 < 0:00:00 , ? it/s ]
   0% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/1  [ 0:00:00 < -:--:-- , ? it/s ]

 INFO     Creating corpus split...                                              


 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:01 < 0:00:00 , ? it/s ]


 WARNING  There were 39 pronunciations in the dictionary that were ignored for  
          containing one of 71 phones not present in the trained acoustic model.
          Please run `mfa validate` to get more details.                        
 INFO     Compiling training graphs...                                          
/Users/jlee/anaconda3/envs/Koach/lib/python3.8/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


KeyboardInterrupt: 